## Language Modelling from scratch using code-mixed Tweets data

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'"

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-g2u9nesu
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-g2u9nesu
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 8.0MB/s 
     |████████████████████████████████| 890kB 31.1MB/s 
  Created wheel for transformers: filename=transformers-4.2.0.dev0-cp36-none-any.whl size=1738082 sha256=ae41b746a4e934de0e030d81b3d600f25e2b842e8486ee60cd72948f23ed450b
  Stored in directory: /tmp/pip-ephem-wheel-cache-v742ewpt/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=abc293aa97e6424d489d17ebff6e18e0b07d9b9cb4957438f0c13e74cca11017
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5

In [ ]:
!nvidia-smi

Tue Jan 12 16:25:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import DistilBertForMaskedLM, DistilBertModel, DistilBertConfig, DistilBertTokenizerFast, Trainer, TrainingArguments

# from nlp import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


In [ ]:
configuration = DistilBertConfig(
                vocab_size=52_000,
                max_position_embeddings=514,
              )
model = DistilBertForMaskedLM(config=configuration)
model.num_parameters()

83503648

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/cleaned_tweets_language_modelling.txt",
    block_size=128,
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 3.11 s, sys: 65.8 ms, total: 3.17 s
Wall time: 1.9 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./sentimix_distillbert",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.555900
1000,5.556700
1500,5.292500
2000,5.073000
2500,4.838100
3000,4.637100
3500,4.483100
4000,4.367100
4500,4.308300


CPU times: user 1h 1min 51s, sys: 39min 7s, total: 1h 40min 58s
Wall time: 1h 41min 9s


TrainOutput(global_step=4740, training_loss=4.976395232767999, metrics={'train_runtime': 6069.8017, 'train_samples_per_second': 0.781, 'total_flos': 10279024675812672, 'epoch': 20.0})

In [ ]:
trainer.save_model("./sentimix_distillbert")

In [ ]:
tokenizer.save_pretrained("./sentimix_distillbert")

('./sentimix_distillbert/tokenizer_config.json',
 './sentimix_distillbert/special_tokens_map.json',
 './sentimix_distillbert/vocab.txt',
 './sentimix_distillbert/added_tokens.json')

In [ ]:
! ls ./sentimix_distillbert

config.json	   special_tokens_map.json  training_args.bin
pytorch_model.bin  tokenizer_config.json    vocab.txt


In [ ]:
! du -sh ./*

2.0M	./cleaned_tweets_language_modelling.txt
20K	./runs
55M	./sample_data
319M	./sentimix_distillbert
319M	./sentimix_distillbert.tar
296M	./sentimix_distillbert.tar.gz


In [ ]:
!tar -czvf ./sentimix_distillbert.tar.gz ./sentimix_distillbert

./sentimix_distillbert/
./sentimix_distillbert/config.json
./sentimix_distillbert/vocab.txt
./sentimix_distillbert/special_tokens_map.json
./sentimix_distillbert/pytorch_model.bin
y
./sentimix_distillbert/training_args.bin
./sentimix_distillbert/tokenizer_config.json




### testing the code mix language model with masked token prediction pipeline



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./sentimix_distillbert",
    tokenizer="./sentimix_distillbert"
)

In [ ]:
fill_mask("Narendra [MASK]")

[{'score': 0.3584403097629547,
  'sequence': '[CLS] narendramo [SEP]',
  'token': 5302,
  'token_str': '##mo'},
 {'score': 0.1295693814754486,
  'sequence': '[CLS] narendra na [SEP]',
  'token': 6583,
  'token_str': 'na'},
 {'score': 0.11362219601869583,
  'sequence': '[CLS] narendradi [SEP]',
  'token': 4305,
  'token_str': '##di'},
 {'score': 0.04874403774738312,
  'sequence': '[CLS] narendra mention [SEP]',
  'token': 5254,
  'token_str': 'mention'},
 {'score': 0.01795562356710434,
  'sequence': '[CLS] narendra mod [SEP]',
  'token': 16913,
  'token_str': 'mod'}]

In [ ]:
fill_mask("bhaijan [MASK]")

[{'score': 0.04731927067041397,
  'sequence': '[CLS] bhaijanhai [SEP]',
  'token': 10932,
  'token_str': '##hai'},
 {'score': 0.03670573979616165,
  'sequence': '[CLS] bhaijan mention [SEP]',
  'token': 5254,
  'token_str': 'mention'},
 {'score': 0.027053413912653923,
  'sequence': '[CLS] bhaijan b [SEP]',
  'token': 1038,
  'token_str': 'b'},
 {'score': 0.02451406791806221,
  'sequence': '[CLS] bhaijanhi [SEP]',
  'token': 4048,
  'token_str': '##hi'},
 {'score': 0.02246043086051941,
  'sequence': '[CLS] bhaijani [SEP]',
  'token': 2072,
  'token_str': '##i'}]

In [ ]:
"Keep such biased opinion to yourself . Don't bother others . Nobody cares . Jai Shree Ram Jai Shree "
fill_mask("Keep such biased [MASK] to yourself")


[{'score': 0.04646363854408264,
  'sequence': '[CLS] keep such biased you to yourself [SEP]',
  'token': 2017,
  'token_str': 'you'},
 {'score': 0.031075648963451385,
  'sequence': '[CLS] keep such biased the to yourself [SEP]',
  'token': 1996,
  'token_str': 'the'},
 {'score': 0.026044048368930817,
  'sequence': '[CLS] keep such biased. to yourself [SEP]',
  'token': 1012,
  'token_str': '.'},
 {'score': 0.024059532210230827,
  'sequence': '[CLS] keep such biased of to yourself [SEP]',
  'token': 1997,
  'token_str': 'of'},
 {'score': 0.019056813791394234,
  'sequence': '[CLS] keep such biased for to yourself [SEP]',
  'token': 2005,
  'token_str': 'for'}]